In [ ]:
import os
from statistics import mode
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, normalize, minmax_scale

x_train_load = pd.read_csv('train.csv')
x_test_load = pd.read_csv('test.csv')
y_train_load = pd.read_csv('train_labels.csv')
y_train = y_train_load.loc[:, y_train_load.columns != 'sequence']
sensor_names = ["sensor_00", "sensor_01", "sensor_02", "sensor_03", "sensor_04", "sensor_05", "sensor_06",
                "sensor_07", "sensor_08", "sensor_09", "sensor_10", "sensor_11", "sensor_12"]
submission = pd.DataFrame(
    columns=["sequence"], data=x_test_load[["sequence"]].groupby(np.arange(len(x_test_load[["sequence"]])) // 60).mean())

x_train = pd.DataFrame()
x_test = pd.DataFrame()

for i in sensor_names:
    x_train[i+"_mean"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).mean()
    x_train[i+"_std"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).std()
    x_train[i+"_max"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).max()
    x_train[i+"_min"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).min()
    x_train[i+"_sum"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).sum()
    x_train[i+"_median"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).median()
    x_train[i+"_q1"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).quantile(0.25)
    x_train[i+"_q3"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).quantile(0.75)

    x_test[i+"_mean"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).mean()
    x_test[i+"_std"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).std()
    x_test[i+"_max"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).max()
    x_test[i+"_min"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).min()
    x_test[i+"_sum"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).sum()
    x_test[i+"_median"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).median()
    x_test[i+"_q1"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).quantile(0.25)
    x_test[i+"_q3"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).quantile(0.75)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

model = tree.DecisionTreeClassifier()
model.fit(x_train,y_train.values.ravel())

print("Fitted")

print("Accuracy: ", np.mean(cross_val_score(
    model, x_train, y_train.values.ravel(), cv=5, n_jobs=-1)))

y_pred = model.predict(x_test)


model.fit(x_train, y_train.values.ravel())
y_pred_train = model.predict(x_train)

print("Average precision score: ", average_precision_score(y_train, y_pred_train))
print("Roc score: ", roc_auc_score(y_train, y_pred_train))


tree.plot_tree(model) #Couldn't get this to work, not sure how much it matters...
submission["state"] = y_pred
submission.to_csv('playground_decision_tree_out.csv', index=False)
print("Done")

C:\Users\Calvin\AppData\Local\Temp/ipykernel_27256/240434733.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  x_train[i+"_sum"] = x_train_load[i].groupby(
C:\Users\Calvin\AppData\Local\Temp/ipykernel_27256/240434733.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  x_train[i+"_median"] = x_train_load[i].groupby(
C:\Users\Calvin\AppData\Local\Temp/ipykernel_27256/240434733.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

Fitted
Accuracy:  0.701478842738822
Average precision score:  1.0
Roc score:  1.0
[Text(99.19287824138311, 214.85142857142856, 'X[17] <= -0.201\ngini = 0.5\nsamples = 25968\nvalue = [12954, 13014]'), Text(50.506063375058524, 209.6742857142857, 'X[18] <= 0.226\ngini = 0.325\nsamples = 8483\nvalue = [6750, 1733]'), Text(28.90260887128503, 204.49714285714285, 'X[32] <= 0.792\ngini = 0.253\nsamples = 6555\nvalue = [5581, 974]'), Text(9.209997242039114, 199.32, 'X[36] <= -0.772\ngini = 0.231\nsamples = 6321\nvalue = [5477, 844]'), Text(3.133947054941853, 194.14285714285714, 'X[38] <= -0.905\ngini = 0.5\nsamples = 268\nvalue = [132, 136]'), Text(2.1011690481996514, 188.96571428571428, 'X[96] <= 0.811\ngini = 0.447\nsamples = 151\nvalue = [100, 51]'), Text(1.8162647704776647, 183.78857142857143, 'X[83] <= 0.348\ngini = 0.393\nsamples = 130\nvalue = [95, 35]'), Text(1.6738126316166715, 178.61142857142858, 'X[36] <= -1.096\ngini = 0.463\nsamples = 96\nvalue = [61, 35]'), Text(0.9259389025964565